In [ ]:
import numpy as np
import meadlib as mead
import cosmology
from scipy import integrate
import matplotlib.pyplot as plt

#import sys
#print(sys.executable)

lin_interpolation = True
log_interpolation = True
lin_table = True
log_table= True

#Import P(k) from CAMB file
k_tab,Pk_tab=cosmology.read_CAMB('data/CAMB_matterpower_z0.dat')

#Convery P(k) to Delta^2(k)
Dk_tab=(4.*np.pi)*Pk_tab*(k_tab**3)/(2.*np.pi)**3

#Plot Delta^2(k)
plt.loglog(k_tab,Dk_tab)
plt.xlabel(r'$k$ / h Mpc$^{-1}$')
plt.ylabel(r'$\Delta^2(k)$')
plt.show()

#Create interpolation function for Delta^2(k)
Dk=mead.log_interp1d(k_tab,Dk_tab,fill_value='extrapolate')

In [ ]:
#Standard integrand
def sigma_integrand(k,R):
    return Dk(k)*(mead.Tophat(k*R)**2)/k

#Logarithmic integrand
def sigma_integrand_log(k,R):
    k=np.exp(k)
    return Dk(k)*(mead.Tophat(k*R)**2)

#Function to calculate the integral
#def sigma(R):
#    kmin=1e-3
#    kmax=1e2
#    #sig2,_=integrate.quad(sigma_integrand, kmin, kmax, args=(R,), limit=50)
#    #sig2,_=integrate.quad(sigma_integrand_log, np.log(kmin), np.log(kmax), args=(R,), limit=50)
#    sig2,_=mead.integrate_quad_log(sigma_integrand, kmin, kmax, args=(R,), limit=50)
#    return np.sqrt(sig2),_

#Set the range in 'R' to plot
#rmin=1e-2
#rmax=10
#nr=100
#R=mead.logspace(rmin,rmax,nr)
R=8.
print('R [Mpc/h]:', R)
print()

#Range for the integration of interpolation function
kmin=k_tab[1]
kmax=k_tab[-1]
print('kmin [h/Mpc]:', kmin)
print('kmax [h/Mpc]:', kmax)
print()

#Integrate interpolation function in linear space
if(lin_interpolation):
    sigma_squared,_=integrate.quad((lambda k, R: Dk(k)*(mead.Tophat(k*R)**2)/k),kmin,kmax,args=(R,))
    print('Linear integral via interpolation function')
    print('sigma:', np.sqrt(sigma_squared))
    print('error:', _)
    print()

#Integrate interpolation function in log space
if(log_interpolation):
    sigma_squared,_=mead.integrate_quad_log(lambda k, R: Dk(k)*(mead.Tophat(k*R)**2)/k,kmin,kmax,args=(R,))
    print('Logarthimic integral via interpolation function')
    print('sigma:', np.sqrt(sigma_squared))
    print('error:', _)
    print()

#Use Simpson's method on the array
if(lin_table):
    inttab=Dk_tab*(mead.Tophat(k_tab*R)**2)/k_tab
    sigma_squared=integrate.simps(inttab,k_tab)
    print('Integral linearly via table')
    print('sigma:', np.sqrt(sigma_squared))
    print()
    
#Use Simpson's method on the array, but logarithmically
if(log_table):
    inttab=Dk_tab*(mead.Tophat(k_tab*R)**2)#/k_tab
    sigma_squared=integrate.simps(inttab,np.log(k_tab))
    print('Integral logarithmically via table')
    print('sigma:', np.sqrt(sigma_squared))
    print()